In [ ]:
!pip install --upgrade langchain langchain-community pdfplumber

In [37]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
import os
import pdfplumber
import re

import base64

encoded = 'Z3NrX09EYmc5aDRDSmtLMXB4Ulc1TWpkV0dkeWIzRllDUXZvV3Q1NzMzWlZmS2JTV1poTExJTGE='


#Extracting text from resume
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        return "\n".join([page.extract_text() or "" for page in pdf.pages])

#Extract fields from text
def extract_resume_data(text):
    name = text.split('\n')[0].strip()
    email_match = re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", text)
    phone_match = re.search(r"(\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}", text)
    return {
        "name": name,
        "email": email_match.group(0) if email_match else "your@email.com",
        "mobile_number": phone_match.group(0) if phone_match else "000-000-0000",
        "linkedin": "https://www.linkedin.com/in/hrithik-sharma-27131214b/",
        "total_experience": "4+",
        "top_impact": "optimizing data pipelines and building scalable dashboards"
    }

# Step 3: Load resume and parse
resume_text = extract_text_from_pdf("HS_DataAnalystresume1.pdf")
data = extract_resume_data(resume_text)

# Step 4: Define Groq prompt
system_template = "You are an AI assistant writing professional cold emails for job applicants."
human_template = """
Write a polished cold email using the following data and my top skills. Do not use placeholders.

Candidate Name: {name}
Experience: {total_experience} years
Top Impact: {top_impact}
Phone: {mobile_number}
LinkedIn: {linkedin}
Recruiter: Jordan
Company: Acme Corp
Job Title: Data Analyst

Write the email with a subject line. Format it as plain text, ready to be emailed.
"""

system_message = SystemMessagePromptTemplate.from_template(system_template)
human_message = HumanMessagePromptTemplate.from_template(human_template)
prompt = ChatPromptTemplate.from_messages([system_message, human_message])

#Generate email using Groq
os.environ["GROQ_API_KEY"] = base64.b64decode(encoded.encode()).decode()
llm = ChatGroq(model="llama-3.1-8b-instant")
chain = LLMChain(prompt=prompt, llm=llm)
response = chain.invoke(data)

#Clean and display
email_raw = response["text"]

#remove stray \n or extra line spacing
email_clean = "\n".join([line.strip() for line in email_raw.split("\n") if line.strip()])
print(email_clean)


Subject: Data Analyst Opportunity at Acme Corp
Dear Jordan,
I came across the Data Analyst position at Acme Corp and was impressed by the company's innovative approach to data-driven decision making. With 4+ years of experience in optimizing data pipelines and building scalable dashboards, I believe I would be a strong fit for this role.
Throughout my career, I've developed a passion for leveraging data to drive business growth and improvement. My expertise in data pipeline optimization has allowed me to increase efficiency, reduce costs, and enhance the overall quality of data. Additionally, my experience in building scalable dashboards has enabled stakeholders to make data-driven decisions with ease.
I'd love the opportunity to discuss my qualifications further and learn more about the Data Analyst role at Acme Corp. Please feel free to reach out to me directly at 734-394-9418 or connect with me on LinkedIn at https://www.linkedin.com/in/hrithik-sharma-27131214b/.
Thank you for consi